In [17]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/tweet_emotions.csv.gz', compression='gzip')

data.groupby("sentiment").nunique()

,tweet_id,content
sentiment,,
anger,110,110
boredom,179,179
empty,827,827
enthusiasm,759,759
fun,1776,1776
happiness,5209,5194
hate,1323,1323
love,3842,3801
neutral,8638,8617
